In [1]:
!pip install fastapi uvicorn faiss-cpu torch transformers pandas numpy ollama


  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
Using cached fastapi-0.115.11-py3-none-any.whl (94 kB)
Using cached uvicorn-0.34.0-py3-none-any.whl (62 kB)
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/13.7 MB 10.1 MB/s eta 0:00:02
   ------------ --------------------------- 4.2/13.7 MB 11.0 MB/s eta 0:00:01
   ------------------ --------------------- 6.3/13.7 MB 10.2 MB/s eta 0:00:01
   --------------------- ------------------ 7.3/13.7 MB 8.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.7/13.7 MB 9.3 MB/s eta 0:00:01
   ---------------------------------- ----- 11.8/13.7 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.7 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 9.0 MB/s eta 0:00:00
Us

In [14]:
!jupyter nbconvert --to script app.ipynb


[NbConvertApp] Converting notebook app.ipynb to script
[NbConvertApp] Writing 4005 bytes to app.py


In [ ]:
import os
import uvicorn
import nest_asyncio
import faiss
import torch
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModel
import ollama
from fastapi.middleware.cors import CORSMiddleware



# --- Initialize FastAPI ---
app = FastAPI()

# --- Load Faiss Index ---
index_path = r"C:\Users\avnex\OneDrive\Desktop\modernbert_faiss_index.bin"  
if os.path.exists(index_path):
    en_index = faiss.read_index(index_path)
    print("✅ Faiss index loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Index file not found at {index_path}")

# --- Load Dataset (Metadata) ---
df = pd.read_csv(r"C:\Users\avnex\OneDrive\Desktop\combined.csv")

# --- Load ModernBERT Model ---
model_id = "nomic-ai/modernbert-embed-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
modernbert_model = AutoModel.from_pretrained(model_id)

# --- Translation Function (Using Ollama Llama3.2) ---
MODEL_NAME = "llama3.2:latest"

def translate_to_english(text: str) -> str:
    """Translates input text to English while preserving semantic meaning."""
    prompt = f"Convert the following text to English while preserving the semantic meaning, don't give an explanation:\n\n{text}"
    response = ollama.chat(
        model=MODEL_NAME, 
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content'].strip()

# --- Embedding Function ---
def get_modernbert_embedding(text: str):
    """Encodes a query using ModernBERT."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = modernbert_model(**inputs)
    emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return emb[0].astype("float32")

# --- Semantic Search Function ---
def semantic_search(query_vector, k=5):
    """Searches the Faiss index for the k nearest neighbors."""
    query_vector = query_vector.reshape(1, -1)
    faiss.normalize_L2(query_vector)
    distances, indices = en_index.search(query_vector, k)
    
    results = df.iloc[indices[0]][["S.No.", "Description", "Class", "Group", "Sub Class"]].to_dict(orient="records")
    
    return results

# --- Define API Request Model ---
class QueryRequest(BaseModel):
    query: str
    k: int = 5

# --- Search Endpoint ---
@app.post("/search")
async def search(request: QueryRequest):
    """Performs a semantic search based on the user's input query."""
    try:
        query_text = translate_to_english(request.query)  # Translate query to English
        query_vector = get_modernbert_embedding(query_text)  # Get embedding
        results = semantic_search(query_vector, request.k)  # Get top-k results
        
        return {"query": request.query, "translated_query": query_text, "results": results}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# --- Root Endpoint ---
@app.get("/")
async def root():
    return {"message": "Welcome to the Semantic Search API!"}
# Add CORS middleware to allow frontend to access the API
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500"],  # Allow requests from your frontend
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)


# --- Run FastAPI Inside Jupyter ---
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=9040)


✅ Faiss index loaded successfully!


INFO:     Started server process [42500]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9040 (Press CTRL+C to quit)


INFO:     127.0.0.1:61463 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61463 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61464 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61494 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61521 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61526 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61543 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61555 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61560 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61577 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61598 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:61604 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53569 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53580 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53587 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53644 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53750 - "POST /search HTTP/1.1" 2